# G-Score Strategy - Investments Committee Quant Sector


In [4]:
from polygon import RESTClient
from collections import defaultdict
import csv
import statistics
import datetime
from dateutil.relativedelta import relativedelta
import ipynb.fs.defs.sec as sec
from tqdm import tqdm
import json

#not easily able to access shared files so resorting to link download instead
#from google.colab import drive
#drive.mount("/content/drive", force_remount=True)

client = RESTClient("xYKWwl2ZEPs4aeN2L4St2B1uuwXrOpfH")

In [5]:
def get_g_scores(query_date: str, financials: dict):
  tickers = {}
  with open('s&p500.csv', mode ='r') as file:
    tickers = {k:v for (k,v) in csv.reader(file)} # convert csv to dict
    
  ticker_data = defaultdict(lambda: {})
  industry_data = defaultdict(lambda: defaultdict(lambda: []*13))

  try:
      client.get_aggs(ticker="AAPL", multiplier=1, timespan='day', from_=query_date, to=query_date)[0].close
  except:
    next_day = datetime.datetime.strptime(query_date, '%Y-%m-%d')
    next_day += relativedelta(days=+1)
    return get_g_scores(next_day.strftime("%Y-%m-%d"), financials)

  for t in tqdm(financials.keys()):
    try:
      earnings_per_share = sec.get_earnings_per_share(financials[t], query_date)
      shares_outstanding = sec.get_shares_outstanding(financials[t], query_date)
      debt = sec.get_total_debt(financials[t], query_date)
      cash = sec.get_cash(financials[t], query_date)
      bvps = sec.get_book_value_per_share(financials[t], query_date)
      net_income = sec.get_net_income(financials[t], query_date)
      total_assets = sec.get_total_assets(financials[t], query_date)
      ebitda = sec.get_ebitda(financials[t], query_date)
      operating_cash_flow = sec.get_operating_cash_flow(financials[t], query_date)
      try:
        r_d = sec.get_research_and_development(financials[t], query_date)
      except:
        r_d = 0

      ticker_data[t]["Price"] = client.get_aggs(ticker=t, multiplier=1, timespan='day', from_=query_date, to=query_date)[0].close
      ticker_data[t]["Volume"] = client.get_aggs(ticker=t, multiplier=1, timespan='day', from_=query_date, to=query_date)[0].volume
      ticker_data[t]["P/E"] = ticker_data[t]["Price"] / earnings_per_share
      ticker_data[t]["EV"] = shares_outstanding * ticker_data[t]["Price"]+ debt - cash
      ticker_data[t]["EBITDA"] = ebitda
      ticker_data[t]["EV/EBITDA"] = ticker_data[t]["EV"] / ticker_data[t]["EBITDA"]
      ticker_data[t]["P/B"] = ticker_data[t]["Price"] / bvps
      ticker_data[t]["ROA"] = net_income / total_assets
      ticker_data[t]["CFROA"] = operating_cash_flow / total_assets 
      ticker_data[t]["MACD"] = client.get_macd(ticker=t, timestamp = query_date).values[0].value
      ticker_data[t]["RSI"] = client.get_rsi(ticker=t, timestamp = query_date).values[0].value
      ticker_data[t]["R&D"] = r_d
      for k, v in ticker_data[t].items(): # add values to industry-specific data
        industry_data[tickers[t]][k].append(v)
    except:
      ticker_data.pop(t, None)

  print(query_date + ": " + str(len(ticker_data.keys())))

  for ind, values in industry_data.items():
    for category, data in values.items():
      industry_data[ind][category] = statistics.median(data)
  
  weightings = {"P/E": .15, "EV/EBITDA": .1, "P/B": .1, "ROA": .175, "CFROA": .175, "MACD": .15, "RSI": .15}
  comparisonTypes = {"P/E": '<', "EV/EBITDA": '<', "P/B": '<', "ROA": '>', "CFROA": '>', "MACD": '>', "RSI": '<'}
  # Dictates how ticker_val and industry_val are compared: ticker_val[P/E] < or > industry_val[P/E]

  def compare(t, category): 
    (ticker_val, industry_val) = (ticker_data[t][category], industry_data[tickers[t]][category])
    return ticker_val > industry_val if comparisonTypes[category] == '>' else ticker_val < industry_val

  g_scores = {t : sum(weight for (category, weight) in weightings.items() if compare(t, category)) 
              for t in ticker_data.keys()}
  
  with open("/Users/kevin/Documents/GitHub/g-score/output/g_scores_" + query_date + ".json", "w") as outfile:
    json.dump(g_scores, outfile)
  with open("/Users/kevin/Documents/GitHub/g-score/financials/ticker_data_" + query_date + ".json", "w") as outfile:
    json.dump(ticker_data, outfile)
  with open("/Users/kevin/Documents/GitHub/g-score/financials/industry_data_" + query_date + ".json", "w") as outfile:
    json.dump(industry_data, outfile)

  return g_scores

In [6]:
def backtest(g_scores_dict, starting_date):
  percent_change = 0
  curr_holdings = {}
  final_date = ''
  prev_date = "00-00-00"
  for curr_date in g_scores_dict.keys():
    curr_change = 0
    if datetime.datetime.strptime(curr_date, '%Y-%m-%d') != starting_date:
      for s in curr_holdings.keys():
        curr_price = client.get_aggs(ticker=s, multiplier=1, timespan='day', from_=curr_date, to=curr_date)[0].close
        curr_change += (curr_price - curr_holdings[s]) / curr_holdings[s]
      curr_change /= len(curr_holdings.keys())
      print(prev_date + " - " + curr_date + ": " + str(round(100 * curr_change, 2)) + "%")
      #SPY_change = (client.get_aggs(ticker=s, multiplier=1, timespan='day', from_=curr_date, to=curr_date)[0].close - client.get_aggs(ticker=s, multiplier=1, timespan='day', from_=prev_date, to=prev_date)[0].close) / client.get_aggs(ticker=s, multiplier=1, timespan='day', from_=prev_date, to=prev_date)[0].close
      #print("SPY: " + str(100 * SPY_change) + "%")
    percent_change += curr_change
    curr_holdings.clear()

    num_stocks = 0
    prev_score = 0
    for stock in reversed(sorted(g_scores_dict[curr_date].items(), key=lambda x:x[1])):
      if stock[1] < prev_score and num_stocks >= 5:
        break
      curr_holdings[stock[0]] = client.get_aggs(ticker=stock[0], multiplier=1, timespan='day', from_=curr_date, to=curr_date)[0].close
      prev_score = stock[1]
      num_stocks += 1
    prev_date = curr_date

    #print("Portfolio Update: " + curr_date)
    #for s in curr_holdings.keys():
    #  print(s + ": " + str(curr_holdings[s]))
    final_date = curr_date
  
  print("Overall P/L: " + str(round(percent_change * 100, 2)))
  spy_start = client.get_aggs(ticker="SPY", multiplier=1, timespan='day', from_=starting_date.strftime('%Y-%m-%d'), to=starting_date.strftime('%Y-%m-%d'))[0].close
  spy_end = client.get_aggs(ticker="SPY", multiplier=1, timespan='day', from_=final_date, to=final_date)[0].close
  print("SPY P/L: " + str(round((spy_end - spy_start) / spy_start * 100, 2)))

In [7]:
def backtest2(g_scores_dict, starting_date):
  tickers = {}
  with open('s&p500.csv', mode ='r') as file:
    tickers = {k:v for (k,v) in csv.reader(file)} # convert csv to dict
  percent_change = 1
  curr_holdings = {}
  final_date = ''
  prev_date = "00-00-00"
  for curr_date in g_scores_dict.keys():
    curr_change = 0
    if datetime.datetime.strptime(curr_date, '%Y-%m-%d') != starting_date:
      no_data = 0
      for s in curr_holdings.keys():
        try:
          curr_price = client.get_aggs(ticker=s, multiplier=1, timespan='day', from_=curr_date, to=curr_date)[0].close
          curr_change += (curr_price - curr_holdings[s]) / curr_holdings[s]
        except:
          no_data += 1
      curr_change /= (len(curr_holdings.keys()) + no_data)
      print(prev_date + " - " + curr_date + ": " + str(round(100 * curr_change, 2)) + "%")
      #SPY_change = (client.get_aggs(ticker=s, multiplier=1, timespan='day', from_=curr_date, to=curr_date)[0].close - client.get_aggs(ticker=s, multiplier=1, timespan='day', from_=prev_date, to=prev_date)[0].close) / client.get_aggs(ticker=s, multiplier=1, timespan='day', from_=prev_date, to=prev_date)[0].close
      #print("SPY: " + str(100 * SPY_change) + "%")
    percent_change *= (1 + curr_change)
    curr_holdings.clear()

    prev_score = 0
    industries = {}
    curr_score = []
    for stock in reversed(sorted(g_scores_dict[curr_date].items(), key=lambda x:x[1])):
      if len(curr_holdings) >= 10 or (prev_score < 0.8 and prev_score != 0):
        break
      if prev_score == 0 or stock[1] == prev_score:
        curr_score.append(stock)
        prev_score = stock[1]
      else:
        if len(curr_score) <= (10 - len(curr_holdings)):
          for s in curr_score:
             if (tickers[s[0]] not in industries.keys()) or (tickers[s[0]] in industries.keys() and industries[tickers[s[0]]] < 3):
              try:
                curr_holdings[s[0]] = client.get_aggs(ticker=s[0], multiplier=1, timespan='day', from_=curr_date, to=curr_date)[0].close
                if tickers[s[0]] in industries.keys():
                  industries[tickers[s[0]]] += 1
                else:
                  industries[tickers[s[0]]] = 1
              except:
                continue
        else:
          RSI_vals = {}
          for s in curr_score:
            try:
              RSI_vals[s[0]] = client.get_rsi(ticker=s[0], timestamp = curr_date).values[0].value
            except:
              continue
          while len(curr_holdings) < 10 and len(RSI_vals) > 0:
            to_add = min(RSI_vals, key=RSI_vals.get)
            if (tickers[to_add] not in industries.keys()) or (tickers[to_add] in industries.keys() and industries[tickers[to_add]] < 3):
              try:
                curr_holdings[to_add] = client.get_aggs(ticker=to_add, multiplier=1, timespan='day', from_=curr_date, to=curr_date)[0].close
                if tickers[to_add] in industries.keys():
                  industries[tickers[to_add]] += 1
                else:
                  industries[tickers[to_add]] = 1
              except:
                RSI_vals.pop(to_add)
                continue
            RSI_vals.pop(to_add)
        curr_score.clear()
        curr_score.append(stock)
        prev_score = stock[1]

    prev_date = curr_date

    print("Portfolio Update: " + curr_date)
    for s in curr_holdings.keys():
      print(s + ": " + str(curr_holdings[s]))
    final_date = curr_date
  
  print("Overall P/L: " + str(round((percent_change - 1) * 100, 2)) + "%")
  spy_start = client.get_aggs(ticker="SPY", multiplier=1, timespan='day', from_=starting_date.strftime('%Y-%m-%d'), to=starting_date.strftime('%Y-%m-%d'))[0].close
  spy_end = client.get_aggs(ticker="SPY", multiplier=1, timespan='day', from_=final_date, to=final_date)[0].close
  print("SPY P/L: " + str(round((spy_end - spy_start) / spy_start * 100, 2)) + "%")

The following code runs the backtest for the last 5 years

In [10]:
tickers = {}
with open('s&p500.csv', mode ='r') as file:
    tickers = {k:v for (k,v) in csv.reader(file)} # convert csv to dict
ticker_financials = {}
for t in tickers.keys():
    try:
        ticker_financials[t] = sec.get_financials(t)
    except:
        print("Could not generate financials for " + t)

Could not generate financials for BRK.B
Could not generate financials for BF.B
Could not generate financials for FRC
Could not generate financials for SBNY
Could not generate financials for VNO


In [ ]:
# create historical data
query_date = datetime.datetime.strptime("2021-12-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2023-03-01", '%Y-%m-%d')
while query_date <= end_date:
    get_g_scores(query_date.strftime("%Y-%m-%d"), ticker_financials)
    query_date += relativedelta(months=+1)

In [11]:
def g_score_files(start_date):
    weightings = {"P/E": .20, "EV/EBITDA": .15, "P/B": .15, "ROA": .175, "CFROA": .175, "MACD": .15}
    comparisonTypes = {"P/E": '<', "EV/EBITDA": '<', "P/B": '<', "ROA": '>', "CFROA": '>', "MACD": '>'}
    # Dictates how ticker_val and industry_val are compared: ticker_val[P/E] < or > industry_val[P/E]
    tickers = {}
    with open('s&p500.csv', mode ='r') as file:
        tickers = {k:v for (k,v) in csv.reader(file)} # convert csv to dict

    with open('/Users/kevin/Documents/GitHub/g-score/financials/ticker_data_' + start_date + '.json') as json_file:
        ticker_data = json.load(json_file)
    with open('/Users/kevin/Documents/GitHub/g-score/financials/industry_data_' + start_date + '.json') as json_file:
        industry_data = json.load(json_file)
    
    def compare(t, category): 
        (ticker_val, industry_val) = (ticker_data[t][category], industry_data[tickers[t]][category])
        return ticker_val > industry_val if comparisonTypes[category] == '>' else ticker_val < industry_val

    g_scores = {t : sum(weight for (category, weight) in weightings.items() if compare(t, category)) 
              for t in ticker_data.keys()}

    return g_scores

In [12]:
print(g_score_files("2013-03-01"))

{'MMM': 0.6499999999999999, 'AOS': 0.8500000000000001, 'ABT': 0.5, 'ACN': 0.15, 'ADM': 0.5, 'ADBE': 0.15, 'AFL': 0.8500000000000001, 'A': 0.8500000000000001, 'APD': 0.32499999999999996, 'ALK': 1.0, 'ALB': 0.3, 'ARE': 0.6499999999999999, 'LNT': 0.8500000000000001, 'MO': 0.32499999999999996, 'AMZN': 0.675, 'AEE': 0.65, 'AEP': 0.32499999999999996, 'AIG': 0.5, 'AMT': 0, 'AWK': 0.32499999999999996, 'ABC': 0.32499999999999996, 'AME': 0.5, 'AMGN': 0.15, 'APH': 0.8500000000000001, 'ADI': 0.7000000000000001, 'ANSS': 0.32499999999999996, 'AON': 0.5, 'AMAT': 0.375, 'AJG': 0.32499999999999996, 'T': 0.5249999999999999, 'AZO': 0.6499999999999999, 'AVY': 0.475, 'BAC': 0.3, 'BAX': 0.5249999999999999, 'BDX': 0.85, 'BBY': 0.65, 'BIIB': 0.5, 'BLK': 0.15, 'BK': 0.5, 'BA': 0.15, 'BWA': 0.44999999999999996, 'BXP': 0.8500000000000001, 'BSX': 0.5, 'BMY': 0.5, 'BR': 0.32499999999999996, 'BRO': 0.7999999999999999, 'CHRW': 0.55, 'CPB': 0.475, 'CAH': 0.3, 'KMX': 0, 'CCL': 0.8500000000000001, 'CE': 0.85, 'CNC': 0.

In [13]:
start_date = datetime.datetime.strptime("2013-03-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2023-03-01", '%Y-%m-%d')

weightings = {"P/E": .15, "EV/EBITDA": .1, "P/B": .1, "ROA": .175, "CFROA": .175, "MACD": .15, "RSI": .15}
comparisonTypes = {"P/E": '<', "EV/EBITDA": '<', "P/B": '<', "ROA": '>', "CFROA": '>', "MACD": '>', "RSI": '<'}
# Dictates how ticker_val and industry_val are compared: ticker_val[P/E] < or > industry_val[P/E]

g_scores_dict = {}
while start_date <= end_date:
    try:
        g_scores_dict[start_date.strftime("%Y-%m-%d")] = g_score_files(start_date.strftime("%Y-%m-%d"))
        start_date += relativedelta(days=+1)
    except:
        start_date += relativedelta(days=+1)
    

print(g_scores_dict)

{'2013-03-01': {'MMM': 0.6499999999999999, 'AOS': 0.8500000000000001, 'ABT': 0.5, 'ACN': 0.15, 'ADM': 0.5, 'ADBE': 0.15, 'AFL': 0.8500000000000001, 'A': 0.8500000000000001, 'APD': 0.32499999999999996, 'ALK': 1.0, 'ALB': 0.3, 'ARE': 0.6499999999999999, 'LNT': 0.8500000000000001, 'MO': 0.32499999999999996, 'AMZN': 0.675, 'AEE': 0.65, 'AEP': 0.32499999999999996, 'AIG': 0.5, 'AMT': 0, 'AWK': 0.32499999999999996, 'ABC': 0.32499999999999996, 'AME': 0.5, 'AMGN': 0.15, 'APH': 0.8500000000000001, 'ADI': 0.7000000000000001, 'ANSS': 0.32499999999999996, 'AON': 0.5, 'AMAT': 0.375, 'AJG': 0.32499999999999996, 'T': 0.5249999999999999, 'AZO': 0.6499999999999999, 'AVY': 0.475, 'BAC': 0.3, 'BAX': 0.5249999999999999, 'BDX': 0.85, 'BBY': 0.65, 'BIIB': 0.5, 'BLK': 0.15, 'BK': 0.5, 'BA': 0.15, 'BWA': 0.44999999999999996, 'BXP': 0.8500000000000001, 'BSX': 0.5, 'BMY': 0.5, 'BR': 0.32499999999999996, 'BRO': 0.7999999999999999, 'CHRW': 0.55, 'CPB': 0.475, 'CAH': 0.3, 'KMX': 0, 'CCL': 0.8500000000000001, 'CE': 

In [14]:
backtest2(g_scores_dict, datetime.datetime.strptime("2013-03-01", '%Y-%m-%d'))

Portfolio Update: 2013-03-01
UDR: 23.97
SWK: 77.59
BEN: 46.9433
TEL: 40.23
HES: 66.54
UNP: 68.285
CVX: 116.9
NKE: 27.41
WDC: 48.18
WST: 30.43
2013-03-01 - 2013-04-01: 4.43%
Portfolio Update: 2013-04-01
SWK: 79.83
TEL: 41.6
MAA: 69.21
UNP: 70.05
WST: 31.665
BEN: 49.8967
WDC: 50.4
NKE: 29.13
HES: 73.54
SJM: 98.95
2013-04-01 - 2013-05-01: 1.83%
Portfolio Update: 2013-05-01
ODFL: 24.7
ALK: 29.865
HES: 70.71
MOS: 60.29
WST: 31.735
AOS: 18.345
BEN: 51.2067
CVX: 120.27
SJM: 102.11
TEL: 42.72
2013-05-01 - 2013-06-03: 3.33%
Portfolio Update: 2013-06-03
NEE: 19.07
LNT: 24.7
UDR: 24.93
SJM: 102.18
SWK: 79.35
CVX: 124.09
TEL: 44.99
TDY: 77.91
UNP: 77.945
MRO: 35.46
2013-06-03 - 2013-07-01: 0.56%
Portfolio Update: 2013-07-01
LNT: 24.83
UNP: 77.71
MRO: 34.83
ITW: 69.55
APH: 19.625
MAA: 67.37
NEE: 20.0575
WDC: 63.06
SJM: 104.08
TEL: 46.41
2013-07-01 - 2013-08-01: 6.23%
Portfolio Update: 2013-08-01
MAA: 66.25
UDR: 25.11
CVX: 126.44
WMT: 78.22
LNT: 26.87
CF: 39.196
TDY: 82.43
HES: 76.04
SWK: 86.98
NEE: